In [33]:
import wandb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.datasets as MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

from PIL import Image

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda:0" if USE_CUDA else 'cpu')
print(DEVICE)

cuda:0


In [3]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: empe8339 (empe8339-gachon-university). Use `wandb login --relogin` to force relogin


True

In [34]:
train_path = "Dataset/MNIST_training.csv"
test_path = "Dataset/MNIST_test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

print(f"train data shape is {train.shape}, test data shape is {test.shape}")
print(test.head())

train data shape is (949, 785), test data shape is (50, 785)
   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      0       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      0       0       0       0       0       0       0       0       0   
3      0       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  

In [5]:
train_y = train[["label"]]
train_x = train.drop(columns = "label")

test_y = test[["label"]]
test_x = test.drop(columns = "label")

##pixel -> image, 각각의 열마다 이미지를 만들어주기, MNIST 이미지가 28에서 끊어주기.

##Train data
os.makedirs("Dataset/train_images", exist_ok=True)
for idx in range(len(train_x)):
    image = train_x.iloc[idx].values.reshape(28,28)
    plt.imsave(f"Dataset/train_images/image_{idx:03d}.png", image, cmap='gray')
    if idx % 100 == 0: 
        print(f"{idx} images processed.")
        
##test Data
os.makedirs("Dataset/test_images", exist_ok=True)

for idx in range(len(test_x)):
    image = test_x.iloc[idx].values.reshape(28,28)
    plt.imsave(f"Dataset/test_images/image_{idx:03d}.png", image, cmap='gray')    
    if idx % 100 == 0:  
        print(f"{idx} images processed.")

0 images processed.
100 images processed.
200 images processed.
300 images processed.
400 images processed.
500 images processed.
600 images processed.
700 images processed.
800 images processed.
900 images processed.
0 images processed.


In [35]:
# Transforms
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),  # Convert PIL Image to Tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize: mean=0.5, std=0.5
])

In [36]:
class MyDataset(Dataset):
    def __init__(self, images_dir, labels, transform=None):
        """
        Args:
            images_dir (str): Directory containing the images.
            labels (pd.DataFrame): DataFrame containing the labels.
            transform (callable, optional): Optional transforms to be applied on an image.
        """
        self.images_dir = images_dir
        self.labels = labels
        self.transform = transform
        self.image_files = sorted(os.listdir(images_dir))  # Ensure consistent order

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Load image
        image_path = os.path.join(self.images_dir, f"image_{idx:03d}.png")
        image = Image.open(image_path).convert("L")  # Convert to grayscale

        # Apply transforms
        if self.transform:
            image = self.transform(image)

        # Get label
        label = self.labels.iloc[idx].values[0]  # Assuming labels are in DataFrame

        return image, label

In [37]:
# Paths
train_images_dir = "Dataset/train_images"
test_images_dir = "Dataset/test_images"

train_labels = pd.read_csv("Dataset/MNIST_training.csv")[["label"]]
test_labels = pd.read_csv("Dataset/MNIST_test.csv")[["label"]]

# Dataset and DataLoader
train_dataset = MyDataset(images_dir=train_images_dir, labels=train_labels, transform=transform)
test_dataset = MyDataset(images_dir=test_images_dir, labels=test_labels, transform=transform)

In [38]:
sweep_config = {
    'name' : "MNIST",
    'method' : 'grid', #gridSearch 
    'metric' : {'goal' : 'maximize', 'name' : 'test_accuracy'},
    'parameters' : {
        'batch_size': {'values': [16, 32]},
        'epochs': {'values': [20, 25]},
        'lr': {'values' : [0.01, 0.001]},
        'optimizer' : {'values' : ['sgd', 'adam', 'rmsprop']}
    }
}

sweep_id = wandb.sweep(sweep_config, project="UNLV_MNIST_CNN")

Create sweep with ID: 0oekq6yj
Sweep URL: https://wandb.ai/empe8339-gachon-university/UNLV_MNIST_CNN/sweeps/0oekq6yj


In [39]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels)
        )
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels

    def forward(self, x):
        residual = x

        # Apply convolution layers
        out = self.conv1(x)
        out = self.conv2(out)

        # Downsample residual if necessary
        if self.downsample:
            residual = self.downsample(x)

        # Add residual and apply activation
        out += residual
        out = self.relu(out)
        return out


In [40]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size = 7, stride = 2, padding = 3),
                                  nn.BatchNorm2d(64),
                                  nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride = 1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None

        # Downsample if stride > 1 or channels don't match
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes

        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        
        return x

In [41]:
"""
epochs = 20
batch_size = 16
learning_rate = 0.01 
"""
num_classes = 10

model = ResNet(ResidualBlock, [2, 2, 2, 2], num_classes=10).to(DEVICE)

##Loss and Optimizer
criterion = nn.CrossEntropyLoss()

In [42]:
def train_CNN():
    with wandb.init() as run:
        config = wandb.config

        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)
        total_step = len(train_loader)

        if config.optimizer == 'sgd':
            optimizer = torch.optim.SGD(model.parameters(), lr = config.lr, weight_decay = 0.001, momentum = 0.9)
        elif config.optimizer == 'rmsprop':
            optimizer = optim.RMSprop(model.parameters(), lr=config.lr)
        elif config.optimizer == 'adam':
            optimizer = optim.Adam(model.parameters(), lr = config.lr)

        for epoch in tqdm(range(config.epochs), desc = "Epochs"):
            for i,(images, labels) in enumerate(train_loader):
                images = images.to(DEVICE)
                labels = labels.to(DEVICE)

                outputs = model(images)
                loss = criterion(outputs, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            print ('Epoch [{}/{}], Loss: {:.4f}' .format(epoch+1, config.epochs, loss.item()))

            wandb.log({
                    'epoch': epoch,
                    'train_loss': loss
            })

            ##test
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader:
                    images = images.to(DEVICE)
                    labels = labels.to(DEVICE)
                    outputs = model(images)
                    val_loss = criterion(outputs, labels)
                    _,predicted = torch.max(outputs.data,1)
                    total+=labels.size(0)
                    correct += (predicted==labels).sum().item()

                accuracy = 100*correct/total

                wandb.log({
                    'test_loss' : val_loss,
                    'test_accuracy' : accuracy
                })
                print("Accuracy of the network on the {} test images {} %".format(50, accuracy))

In [43]:
wandb.agent(sweep_id, function = train_CNN)

wandb: Agent Starting Run: w8lsnyy4 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 20
wandb: 	lr: 0.01
wandb: 	optimizer: sgd


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch [1/20], Loss: 2.0371


Epochs:   5%|▌         | 1/20 [00:04<01:22,  4.32s/it]

Accuracy of the network on the 50 test images 76.0 %


...

Epoch [24/25], Loss: 0.0000
Accuracy of the network on the 50 test images 98.0 %


Epochs: 100%|██████████| 25/25 [00:48<00:00,  1.93s/it]

Epoch [25/25], Loss: 0.0000
Accuracy of the network on the 50 test images 98.0 %


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
test_accuracy,████████████▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,▁▃▃█▅▃▅▅▃▅▅▅▃▁▃▃▃▃▃▃▃▅▅▅▅
train_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁
epoch,24
test_accuracy,98
test_loss,0.0
train_loss,0.0


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
